In [ ]:
import keras
import numpy as np
from sklearn import preprocessing
from keras.layers import Dense, Input, Embedding, Lambda, Layer, Multiply, Dropout, Dot
from keras.models import Model, Sequential
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras import backend as K
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import pandas as pd

In [ ]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

In [ ]:
def load_embeddings(filename, max_vocab_size):

    vocab={}
    embeddings=[]
    with open(filename) as file:
        
        cols=file.readline().split(" ")
        num_words=int(cols[0])
        size=int(cols[1])
        embeddings.append(np.zeros(size))  # 0 = 0 padding if needed
        embeddings.append(np.zeros(size))  # 1 = UNK
        vocab["_0_"]=0
        vocab["_UNK_"]=1
        
        for idx,line in enumerate(file):

            if idx+2 >= max_vocab_size:
                break

            cols=line.rstrip().split(" ")
            val=np.array(cols[1:])
            word=cols[0]
            
            embeddings.append(val)
            vocab[word]=idx+2

    return np.array(embeddings), vocab, size

In [ ]:
def read_data(filename, vocab):
    X=[]
    Y=[]
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            label=cols[1]
            # assumes text is already tokenized
            text=cols[1].split(" ")
            X.append(text)
            Y.append(label)
    return X, Y

In [ ]:
def get_word_ids(docs, vocab, max_length=1000):
    
    doc_ids=[]
    
    for doc in docs:
        wids=[]

        for token in doc[:max_length]:
            val = vocab[token.lower()] if token.lower() in vocab else 1
            wids.append(val)
        
        # pad each document to constant width
        for i in range(len(wids),max_length):
            wids.append(0)

        doc_ids.append(wids)

    return np.array(doc_ids)

In [ ]:
embeddings, vocab, embedding_size=load_embeddings("/content/drive/MyDrive/Event_Extraction/embeddings/embeddings.txt", 10000)

In [ ]:
directory="/content/drive/MyDrive/Event_Extraction/files"

In [ ]:
trainText, trainY = read_data("%s/train_answer_keys.txt" % directory, vocab)
devText, devY=read_data("%s/val_answer_keys.txt" % directory, vocab)

In [ ]:
trainX = get_word_ids(trainText, vocab, max_length=200)
devX = get_word_ids(devText, vocab, max_length=200)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(trainY)
Y_train=np.array(le.transform(trainY))
Y_dev=np.array(le.transform(devY))

First, let's try a simple model that represents a document by averaging the embeddings for the words it contains. We'll again use appropriate masking to accommodate zero-padded sequences.

In [ ]:
from keras import layers
class MaskedAveragePooling1D(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(MaskedAveragePooling1D, self).__init__(**kwargs)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.repeat(mask, x.shape[-1])
            mask = tf.transpose(mask, [0,2,1])
            # zero out the elements of x that are masked
            x = x * mask
            
        # sum the modified input, but normalize only over the number of non-masked time steps
        return K.sum(x, axis=1) / K.sum(mask, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[2])

In [ ]:
def get_embedding_average(embeddings):

    vocab_size, word_embedding_dim=embeddings.shape
    
    word_sequence_input = Input(shape=(None,), dtype='int32')
    
    word_embedding_layer = Embedding(vocab_size,
                                    word_embedding_dim,
                                    weights=[embeddings],
                                    mask_zero=True,
                                    trainable=False)

    
    embedded_sequences = word_embedding_layer(word_sequence_input)
    x=MaskedAveragePooling1D()(embedded_sequences)
    
    predictions=Dense(1, activation="sigmoid")(x)

    model = Model(inputs=word_sequence_input, outputs=predictions)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [ ]:
embedding_model=get_embedding_average(embeddings)
print (embedding_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 300)         3000000   
                                                                 
 masked_average_pooling1d (M  (None, 300)              0         
 askedAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 3,000,301
Trainable params: 301
Non-trainable params: 3,000,000
_________________________________________________________________
None


In [ ]:
model=embedding_model

modelName="embedding_model.hdf5"
checkpoint = ModelCheckpoint(modelName, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

model.fit(trainX, Y_train, 
            validation_data=(devX, Y_dev),
            epochs=30, batch_size=128,
            callbacks=[checkpoint])

Epoch 1/30
57/57 [==============================] - 5s 75ms/step - loss: -1.1745 - acc: 0.0813 - val_loss: -3.2315 - val_acc: 0.0821
Epoch 2/30
 2/57 [>.............................] - ETA: 3s - loss: -3.4960 - acc: 0.1055

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


57/57 [==============================] - 5s 87ms/step - loss: -5.0965 - acc: 0.0824 - val_loss: -7.1616 - val_acc: 0.0821
Epoch 3/30
57/57 [==============================] - 5s 80ms/step - loss: -8.9577 - acc: 0.0824 - val_loss: -11.0480 - val_acc: 0.0821
Epoch 4/30
57/57 [==============================] - 4s 76ms/step - loss: -12.8100 - acc: 0.0824 - val_loss: -14.9509 - val_acc: 0.0821
Epoch 5/30
57/57 [==============================] - 4s 77ms/step - loss: -16.6567 - acc: 0.0824 - val_loss: -18.8356 - val_acc: 0.0821
Epoch 6/30
57/57 [==============================] - 4s 77ms/step - loss: -20.5147 - acc: 0.0824 - val_loss: -22.7533 - val_acc: 0.0821
Epoch 7/30
57/57 [==============================] - 4s 77ms/step - loss: -24.3864 - acc: 0.0824 - val_loss: -26.6526 - val_acc: 0.0821
Epoch 8/30
57/57 [==============================] - 4s 75ms/step - loss: -28.2651 - acc: 0.0824 - val_loss: -30.5638 - val_acc: 0.0821
Epoch 9/30
57/57 [==============================] - 6s 113ms/step - l

In [ ]:
class AttentionLayerMasking(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(AttentionLayerMasking, self).__init__(**kwargs)


    def build(self, input_shape):
        input_embedding_dim=input_shape[-1]
        
        self.kernel = self.add_weight(name='kernel', 
                            shape=(input_embedding_dim,1),
                            initializer='uniform',
                            trainable=True)
        super(AttentionLayerMasking, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        
        # dot product 
        x=K.dot(x, self.kernel)
        # exponentiate
        x=K.exp(x)
        
        # zero out elements that are masked
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, axis=-1)
            x = x * mask
        
        # normalize by sum
        x /= K.sum(x, axis=1, keepdims=True)
        x=K.squeeze(x, axis=2)

        return x

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1])

In [ ]:
def get_embedding_with_attention_masking(embeddings):

    vocab_size, word_embedding_dim=embeddings.shape
    
    word_sequence_input = Input(shape=(None,), dtype='int32')
    
    word_embedding_layer = Embedding(vocab_size,
                                    word_embedding_dim,
                                    weights=[embeddings], 
                                    mask_zero=True,
                                    trainable=False)

    
    embedded_sequences = word_embedding_layer(word_sequence_input)
    
    #transform each word embedding into a new vector to use for measuring its importance
    attention_key_dim=300
    attention_input=Dense(attention_key_dim, activation='tanh')(embedded_sequences)

    # pass those transformed inputs through an attention layer, getting back a normalized
    # attention value a_i for each token i; \forall i, 0 <= a_i <= 1; for a document with N words, 
    # \sum_{i=0}^N a_i = 1
    
    attention_output = AttentionLayerMasking(word_embedding_dim, name="attention")(attention_input)
    
    # now let's multiply those attention weights by original inputs to get a weighted average over them
    document_representation = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=1), name='dot')([attention_output,embedded_sequences])

    x=Dense(1, activation="sigmoid")(document_representation)

    model = Model(inputs=word_sequence_input, outputs=x)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model


In [ ]:
embedding_attention_model = get_embedding_with_attention_masking(embeddings)
print (embedding_attention_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    3000000     ['input_2[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, None, 300)    90300       ['embedding_1[0][0]']            
                                                                                                  
 attention (AttentionLayerMaski  (None, None)        300         ['dense_1[0][0]']                
 ng)                                                                                        

In [ ]:
model = embedding_attention_model

modelName="embedding_attention_model.hdf5"
checkpoint = ModelCheckpoint(modelName, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

model.fit(trainX, Y_train, 
            validation_data=(devX, Y_dev),
            epochs=30, batch_size=128,
            callbacks=[checkpoint])

Epoch 1/30
57/57 [==============================] - ETA: 0s - loss: -1.2559 - acc: 0.0820

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


NotImplementedError: ignored

In [ ]:
# load the best saved model

model=embedding_attention_model
model.load_weights("embedding_attention_model.hdf5")

ValueError: ignored

adding attention to that simple model to learn a weighted average over words---giving more weight to words in the document that are more important for representing the document for the purpose of this classification

In [ ]:
def analyze(model, doc):
    
    words=doc.split(" ")
    text = get_word_ids([words], vocab, max_length=len(words))
   
    inp = model.input                                    
    outputs = [layer.output for layer in model.layers[1:]]       
    functor = K.function([inp, K.learning_phase()], outputs) 

    test = text[0]
    orig=words
    attention_weights=[]
    test=test.reshape((1,len(words)))
    layer_outs = functor([test, 0.])

    # in this model, attention is the third layer
    attention_layer=layer_outs[2]
    
    for i in range(len(words)):
        val=attention_layer[0,i]
        attention_weights.append(val)
        print ("%.3f\t%s" % (val, orig[i]))
        
    df = pd.DataFrame({'words':orig, 'attention':attention_weights})
    ax = df.plot.bar(x='words', y='attention', figsize=(10,4))